<font color="red"> WIP. More in-depth elaboration starting from Phase 1: Connect required</font>

# Running a Federated Cycle with Synergos

This tutorial aims to give you an understanding of how to use the synergos package to run a full federated learning cycle. 

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a TTP and 2 Workers. 

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model

## Initiating the docker containers

Before we begin, we have to start the docker containers.

Firstly, follow the instructions on __\<path to pysyft_ttp readme\>__ and __\<path to pysyft_worker readme\>__ on how to build the required docker images for TTP and Workers.

Next, in <u>separate</u> CLI terminals, run the following command:

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

**Worker 1**

```docker run -v <directory heart_disease/data1>:/worker/data -v <directory heart_disease/outputs_1>:/worker/outputs --name worker_1 worker:pysyft_demo```

**Worker 2**

```docker run -v <directory heart_disease/data2>:/worker/data -v <directory heart_disease/outputs_2>:/worker/outputs --name worker_2 worker:pysyft_demo```

**TTP**

```docker run -p 0.0.0.0:5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v <directory /heart_disease/mlflow_test>:/ttp/mlflow --name ttp --link worker_1 --link worker_2 ttp:pysyft_demo```


Once ready, for each terminal, you should see that a Flask app is running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

## Configuration

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- worker_1 address: 172.17.0.2
- worker_2 address: 172.17.0.3
- ttp address: 172.17.0.4

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

from synergos import Driver

host = "192.168.1.198" #"0.0.0.0" # Different for Windows users
port = 5000

# Initiate Driver
driver = Driver(host=host, port=port)

In [1]:
from synergos import Driver

host = "localhost" # Different for Windows users
port = 5000

# Initiate Driver
driver = Driver(host=host, port=port)

## Phase 1: Connect

Submitting TTP & Participant metadata

In [2]:
collab_task = driver.collaborations
collab_task.create("c2")

{'data': {'doc_id': '1',
  'kind': 'Collaboration',
  'key': {'collab_id': 'c2'},
  'relations': {'Project': [],
   'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []}},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'collaborations.post',
 'params': {}}

#### 1A. TTP controller creates a project

In [3]:
driver.projects.create(
    collab_id="c2",
    project_id="test_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': [],
        'tier_3': []
    }
)

{'data': {'doc_id': '1',
  'kind': 'Project',
  'key': {'collab_id': 'c2', 'project_id': 'test_project'},
  'relations': {'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'action': 'classify',
  'incentives': {'tier_3': [], 'tier_2': [], 'tier_1': []}},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'projects.post',
 'params': {'collab_id': 'c2'}}

#### 1B. TTP controller creates an experiment

In [4]:
driver.experiments.create(
    collab_id="c2",
    project_id="test_project",
    expt_id="test_experiment",
    model=[
        {
            "activation": None,
            "is_input": True,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 13, 
                "out_features": 10 
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "ReLU",
            "structure": {}
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 10, 
                "out_features": 10 
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "ReLU",
            "structure": {}
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 10, 
                "out_features": 5 
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "ReLU",
            "structure": {}
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 5, 
                "out_features": 8 
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "ReLU",
            "structure": {}
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 8, 
                "out_features": 1 
            }
        }
#         {
#             "activation": None,
#             "is_input": False,
#             "l_type": "Sigmoid",
#             "structure": {}
#         },
    ]
)

{'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'collab_id': 'c2', 'project_id': 'test_project'},
 'data': {'created_at': '2021-06-09 09:51:14 N',
  'key': {'collab_id': 'c2',
   'expt_id': 'test_experiment',
   'project_id': 'test_project'},
  'model': [{'activation': None,
    'is_input': True,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 13, 'out_features': 10}},
   {'activation': None, 'is_input': False, 'l_type': 'ReLU', 'structure': {}},
   {'activation': None,
    'is_input': False,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 10, 'out_features': 10}},
   {'activation': None, 'is_input': False, 'l_type': 'ReLU', 'structure': {}},
   {'activation': None,
    'is_input': False,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 10, 'out_features': 5}},
   {'activation': None, 'is_input': False, 'l_type': 'ReLU', 'structure': {}},
   {'activation': None,
    'is

#### 1C. TTP controller creates a run

In [5]:
driver.runs.create(
    collab_id="c2",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    rounds=5, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss",
    algorithm="FedGKT",
    whether_distill_on_the_server=True,
    L=4,
    temperature=3,
    lr_scheduler='ReduceLROnPlateau',
    optimizer='Adam',
    server_epochs=3,
    alpha=1
)

{'data': {'doc_id': '1',
  'kind': 'Run',
  'key': {'collab_id': 'c2',
   'project_id': 'test_project',
   'expt_id': 'test_experiment',
   'run_id': 'test_run'},
  'relations': {'Model': [], 'Validation': [], 'Prediction': []},
  'rounds': 5,
  'epochs': 1,
  'lr': 0.001,
  'lr_decay': 0.1,
  'weight_decay': 0.0,
  'seed': 42,
  'precision_fractional': 5,
  'mu': 0.1,
  'l1_lambda': 0.0,
  'l2_lambda': 0.0,
  'base_lr': 0.0005,
  'max_lr': 0.005,
  'patience': 10,
  'delta': 0.0},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'runs.post',
 'params': {'collab_id': 'c2',
  'project_id': 'test_project',
  'expt_id': 'test_experiment'}}

#### 1D. Participants registers their servers' configurations

In [6]:
driver.participants.create(participant_id='test_participant_1')
driver.participants.create(participant_id='test_participant_2')

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'test_participant_2'},
  'relations': {'Registration': [], 'Tag': []},
  'id': 'test_participant_2'},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'participants.post',
 'params': {}}

In [7]:
registration_task = driver.registrations

registration_task.add_node(
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

'node_0'

In [8]:
registration_task.create(
    collab_id="c2",
    project_id="test_project",
    participant_id="test_participant_1",
    role="host"
)

{'data': {'doc_id': '1',
  'kind': 'Registration',
  'key': {'collab_id': 'c2',
   'project_id': 'test_project',
   'participant_id': 'test_participant_1'},
  'relations': {'Tag': []},
  'collaboration': {'catalogue_host': None,
   'catalogue_port': None,
   'logger_host': None,
   'logger_ports': {'sysmetrics': None,
    'director': None,
    'ttp': None,
    'worker': None},
   'meter_host': None,
   'meter_port': None,
   'mlops_host': None,
   'mlops_port': None,
   'mq_host': None,
   'mq_port': None,
   'ui_host': None,
   'ui_port': None},
  'project': {'incentives': {'tier_3': [], 'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'test_participant_1'},
  'role': 'host',
  'n_count': 1,
  'node_0': {'host': '172.17.0.2',
   'port': 8020,
   'log_msgs': True,
   'verbose': True,
   'f_port': 5000}},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {'collab_id': 'c2',
  'project_id': 'test_project',
  'parti

In [9]:
registration_task.add_node(
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

'node_0'

In [10]:
registration_task.create(
    collab_id="c2",
    project_id="test_project",
    participant_id="test_participant_2",
    role="guest"
)

{'data': {'doc_id': '2',
  'kind': 'Registration',
  'key': {'collab_id': 'c2',
   'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'relations': {'Tag': []},
  'collaboration': {'catalogue_host': None,
   'catalogue_port': None,
   'logger_host': None,
   'logger_ports': {'sysmetrics': None,
    'director': None,
    'ttp': None,
    'worker': None},
   'meter_host': None,
   'meter_port': None,
   'mlops_host': None,
   'mlops_port': None,
   'mq_host': None,
   'mq_port': None,
   'ui_host': None,
   'ui_port': None},
  'project': {'incentives': {'tier_3': [], 'tier_2': [], 'tier_1': []},
   'start_at': None},
  'participant': {'id': 'test_participant_2'},
  'role': 'guest',
  'n_count': 1,
  'node_0': {'host': '172.17.0.3',
   'port': 8020,
   'log_msgs': True,
   'verbose': True,
   'f_port': 5000}},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {'collab_id': 'c2',
  'project_id': 'test_project',
  'part

In [11]:
'''
driver.participants.create(
    participant_id="test_participant_1",
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

driver.participants.create(
    participant_id="test_participant_2",
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)
'''

'\ndriver.participants.create(\n    participant_id="test_participant_1",\n    host=\'172.17.0.2\',\n    port=8020,\n    f_port=5000,\n    log_msgs=True,\n    verbose=True\n)\n\ndriver.participants.create(\n    participant_id="test_participant_2",\n    host=\'172.17.0.3\',\n    port=8020,\n    f_port=5000,\n    log_msgs=True,\n    verbose=True\n)\n'

#### 1E. Participants registers their role in a specific project

In [12]:
'''
driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_1",
    role="guest"
)

driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_2",
    role="host"
)
'''

'\ndriver.registrations.create(\n    project_id="test_project",\n    participant_id="test_participant_1",\n    role="guest"\n)\n\ndriver.registrations.create(\n    project_id="test_project",\n    participant_id="test_participant_2",\n    role="host"\n)\n'

#### 1F. Participants registers their tags for a specific project

In [13]:
driver.tags.create(
    collab_id="c2",
    project_id="test_project",
    participant_id="test_participant_1",
    train=[
        ["non_iid_1"]
    ],
    evaluate=[["iid_1"]]
)

driver.tags.create(
    collab_id="c2",
    project_id="test_project",
    participant_id="test_participant_2",
    train=[["non_iid_2"]]
)

{'data': {'doc_id': '2',
  'kind': 'Tag',
  'key': {'collab_id': 'c2',
   'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'train': [['non_iid_2']],
  'evaluate': [],
  'predict': []},
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'tag.post',
 'params': {'collab_id': 'c2',
  'project_id': 'test_project',
  'participant_id': 'test_participant_2'}}

## Phase 2:
Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [14]:
driver.alignments.create(collab_id="c2", project_id="test_project")

{'data': [{'doc_id': '1',
   'kind': 'Alignment',
   'key': {'collab_id': 'c2',
    'project_id': 'test_project',
    'participant_id': 'test_participant_2'},
   'train': {'X': [], 'y': []},
   'evaluate': {'X': [], 'y': []},
   'predict': {'X': [], 'y': []}},
  {'doc_id': '2',
   'kind': 'Alignment',
   'key': {'collab_id': 'c2',
    'project_id': 'test_project',
    'participant_id': 'test_participant_1'},
   'train': {'X': [], 'y': []},
   'evaluate': {'X': [], 'y': []},
   'predict': {'X': [], 'y': []}}],
 'apiVersion': '0.2.0',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'collab_id': 'c2', 'project_id': 'test_project'}}

#### 2B. Trigger training across the federated grid

In [15]:
model_resp = driver.models.create(
    collab_id="c2",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

#### 2C. Trigger hyperparameter optimisation across the grid

## Phase 3: EVALUATE 
Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [17]:
driver.validations.create(
    collab_id='c2',
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Validation',
   'key': {'participant_id': 'test_participant_1',
    'collab_id': 'c2',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'evaluate': {'statistics': {'accuracy': [0.4254215304798962,
      0.788586251621271,
      0.8391699092088197,
      0.8404669260700389,
      0.9571984435797666],
     'roc_auc_score': [0.7368551450751527,
      0.38828907006780755,
      0.4647878546143491,
      0.3779985948007628,
      0.5724726944239139],
     'pr_auc_score': [0.5863141821038254,
      0.1821355880724637,
      0.16043133176378976,
      0.15628833085577729,
      0.0467752200155003],
     'f_score': [0.5969062784349408, 0.0, 0.0, 0.0, 0.0],
     'TPRs': [1.0, 0.0, 0.0, 0.0, 0.0],
     'TNRs': [0.0, 1.0, 1.0, 1.0, 1.0],
     'PPVs': [0.4254215304798962, 0.0, 0.0, 0.0, 0.0],
     'NPVs': [0.0,
      0.788586251621271,
      0.8391699092088197,
      0.8404669260700389,
      0.9571984435797666]

#### 3B. Perform prediction(s) of combination(s)

In [18]:
driver.predictions.create(
    collab_id="c2",
    tags={"test_project": [["iid_1"]]},
    participant_id="test_participant_1",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Prediction',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'predict': {'statistics': {'accuracy': [0.4254215304798962,
      0.788586251621271,
      0.8391699092088197,
      0.8404669260700389,
      0.9571984435797666],
     'roc_auc_score': [0.7368551450751527,
      0.38828907006780755,
      0.4647878546143491,
      0.3779985948007628,
      0.5724726944239139],
     'pr_auc_score': [0.5863141821038254,
      0.1821355880724637,
      0.16043133176378976,
      0.15628833085577729,
      0.0467752200155003],
     'f_score': [0.5969062784349408, 0.0, 0.0, 0.0, 0.0],
     'TPRs': [1.0, 0.0, 0.0, 0.0, 0.0],
     'TNRs': [0.0, 1.0, 1.0, 1.0, 1.0],
     'PPVs': [0.4254215304798962, 0.0, 0.0, 0.0, 0.0],
     'NPVs': [0.0,
      0.788586251621271,
      0.8391699092088197,
      0.8404669260700389,
      0.9571984435797666],
     'FPRs': [1.0, 0.0